In [1]:
import os

dossier = r"C:\Users\bough\OneDrive\Bureau\TP_archi_moder\casimage_FR"  

# Lister tous les fichiers du dossier
fichiers = os.listdir(dossier)

# Filtrer uniquement les fichiers qui se terminent par .xml (insensible à la casse)
fichiers_xml = [f for f in fichiers if f.lower().endswith('.xml')]

# Afficher le nombre de fichiers XML
print(f"Nombre de fichiers XML dans le dossier : {len(fichiers_xml)}")


Nombre de fichiers XML dans le dossier : 1691


In [2]:
# -------------------------------
# Importation des bibliothèques
# -------------------------------
import os              # Pour interagir avec le système de fichiers (dossiers et fichiers)
import re              # Pour le nettoyage de texte avec des expressions régulières
import pandas as pd    # Pour créer et manipuler des tableaux et exporter en CSV
from lxml import etree # Pour parser (analyser) les fichiers XML
from tqdm import tqdm  # Pour afficher une barre de progression lors des boucles

# -------------------------------
# Fonctions utilitaires
# -------------------------------

def clean_xml_text(text):
    """
    Nettoie le texte XML pour éviter les erreurs de parsing.
    - Remplace les caractères '&' malformés par '&amp;'
    - Supprime les caractères non imprimables
    - Supprime les caractères spéciaux invalides
    - Supprime les espaces entre les balises
    """
    # Corrige les & qui ne sont pas des entités valides
    text = re.sub(r'&(?!amp;|lt;|gt;|quot;|apos;)', '&amp;', text)
    # Supprime les caractères ASCII de contrôle (hors tabulation, saut de ligne, retour chariot)
    text = re.sub(r'[\x00-\x08\x0b\x0c\x0e-\x1f]', '', text)
    # Supprime les caractères invalides pour XML
    text = re.sub(r'[^\x09\x0A\x0D\x20-\x7E\x80-\xFF]', '', text)
    # Supprime les espaces entre les balises (><)
    text = re.sub(r'>\s+<', '><', text)
    return text

def parse_xml_safe(xml_content):
    """
    Parse le contenu XML en utilisant lxml avec récupération automatique
    des erreurs pour éviter que le script plante si le XML est mal formé.
    """
    parser = etree.XMLParser(recover=True)  # Permet de récupérer les erreurs
    return etree.fromstring(xml_content.encode("utf-8"), parser=parser)

def read_file_safe(path):
    """
    Lit un fichier XML en essayant d'abord l'encodage utf-8,
    puis latin-1 si utf-8 échoue.
    """
    try:
        with open(path, "r", encoding="utf-8") as f:
            return f.read()
    except UnicodeDecodeError:
        with open(path, "r", encoding="latin-1") as f:
            return f.read()

# -------------------------------
# Paramètres
# -------------------------------
folder = r"C:\Users\bough\OneDrive\Bureau\TP_archi_moder\casimage_FR"  # Dossier contenant les fichiers XML
output_csv = "casimage_nettoye.csv"  # Nom du fichier CSV de sortie

# -------------------------------
# Étape 1 : Détection de toutes les balises
# -------------------------------
all_tags = set()  # Ensemble pour stocker toutes les balises uniques
files = [f for f in os.listdir(folder) if f.lower().endswith(".xml")]  # Liste de tous les fichiers XML du dossier

# Boucle sur tous les fichiers pour détecter les balises
for file in tqdm(files, desc="Détection des balises"):
    path = os.path.join(folder, file)  # Chemin complet du fichier
    content = read_file_safe(path)     # Lire le contenu du fichier en toute sécurité
    content = clean_xml_text(content)  # Nettoyer le texte
    try:
        root = parse_xml_safe(content)  # Parser le XML
        for elem in root.iter():        # Parcourt tous les éléments du XML
            all_tags.add(elem.tag)      # Ajoute le nom de la balise à l'ensemble
    except Exception as e:
        print(f"⚠️ Erreur parsing {file}: {e}")  # Affiche si un fichier est corrompu

all_tags = sorted(list(all_tags))  # Trie les balises par ordre alphabétique

# -------------------------------
# Étape 2 : Extraction des données
# -------------------------------
data_rows = []  # Liste pour stocker toutes les lignes (dictionnaires) du CSV

# Boucle sur tous les fichiers pour extraire les données
for file in tqdm(files, desc="Extraction des données"):
    path = os.path.join(folder, file)
    content = read_file_safe(path)
    content = clean_xml_text(content)
    try:
        root = parse_xml_safe(content)
        row = {tag: "" for tag in all_tags}  # Initialise un dictionnaire vide avec toutes les balises
        for elem in root.iter():            # Parcourt toutes les balises du fichier XML
            if elem.text:
                row[elem.tag] = elem.text.strip()  # Remplit la valeur du dictionnaire
        data_rows.append(row)  # Ajoute la ligne à la liste des données
    except Exception as e:
        print(f"⚠️ Fichier corrompu : {file} ({e})")  # Message si le fichier est malformé

# -------------------------------
# Étape 3 : Création du CSV
# -------------------------------
df = pd.DataFrame(data_rows)  # Crée un DataFrame pandas à partir de la liste des dictionnaires
df.to_csv(output_csv, index=False, encoding="utf-8-sig")  # Export CSV
print(f"✅ CSV généré avec {len(df)} lignes et {len(all_tags)} colonnes")


Extraction des données: 100%|████████████████████████████████████████████████████| 1691/1691 [00:00<00:00, 2367.81it/s]


✅ CSV généré avec 1691 lignes et 48 colonnes
